# **Welcome to the Notebook**

### Let's mount the google drive 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Task 1 : 
Installing pyspark module

In [2]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 8.9 kB/s 
     |████████████████████████████████| 198 kB 48.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=341c89a2e10c24c32d88fbfd3067c1608f188b56d750bf7236b750be51f090e0
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


Importing the modules 

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, desc , col, max 
import matplotlib.pyplot as plts

creating spark session

In [4]:
spark = SparkSession.builder.appName('spark_app').getOrCreate()

# Task 2 : 
importing the *Listenings.csv* file: 

In [6]:
listening_csv_path = '/content/drive/MyDrive/dataset/listenings.csv'
listening_df = spark.read.format('csv').option('inferSchema',True).option(
'header', True).load(listening_csv_path)

let's check the data:

In [7]:
listening_df.show(4)

+-----------+-------------+--------------------+--------+-----------+
|    user_id|         date|               track|  artist|      album|
+-----------+-------------+--------------------+--------+-----------+
|000Silenced|1299680100000|           Price Tag|Jessie J|Who You Are|
|000Silenced|1299679920000|Price Tag (Acoust...|Jessie J|  Price Tag|
|000Silenced|1299679440000|Be Mine! (Ballad ...|   Robyn|   Be Mine!|
|000Silenced|1299679200000|            Acapella|   Kelis|   Acapella|
+-----------+-------------+--------------------+--------+-----------+
only showing top 4 rows



let's delete useless columns:

In [8]:
listening_df = listening_df.drop('date')

drop the null rows:

In [9]:
listening_df = listening_df.na.drop()

let's check the dataset again:

In [10]:
listening_df.show(3)

+-----------+--------------------+--------+-----------+
|    user_id|               track|  artist|      album|
+-----------+--------------------+--------+-----------+
|000Silenced|           Price Tag|Jessie J|Who You Are|
|000Silenced|Price Tag (Acoust...|Jessie J|  Price Tag|
|000Silenced|Be Mine! (Ballad ...|   Robyn|   Be Mine!|
+-----------+--------------------+--------+-----------+
only showing top 3 rows



let's see the schema: 

In [11]:
listening_df.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- track: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- album: string (nullable = true)



let's see the shape of our dataframe: 

In [13]:
shape  = ( listening_df.count(), len(   listening_df.columns) )
print(shape)

(13758905, 4)


# Task 3:

**Query #0:**
select two columns: track and artist

In [14]:
q0 = listening_df.select('artist','track')
q0.show(3)

+--------+--------------------+
|  artist|               track|
+--------+--------------------+
|Jessie J|           Price Tag|
|Jessie J|Price Tag (Acoust...|
|   Robyn|Be Mine! (Ballad ...|
+--------+--------------------+
only showing top 3 rows



**Query #1**:

Let's find all of the records of those users who have listened to ***Rihanna***

In [16]:
q1 = listening_df.select('*').filter(listening_df.artist == 'Rihanna')
q1.show(4)

+-----------+--------------------+-------+--------------------+
|    user_id|               track| artist|               album|
+-----------+--------------------+-------+--------------------+
|000Silenced|Don't Stop The Mu...|Rihanna|Addicted 2 Bassli...|
|000Silenced|           Disturbia|Rihanna|Good Girl Gone Ba...|
|00williamsl|   Hatin On The Club|Rihanna|              Random|
|00williamsl|   Hatin On The Club|Rihanna|              Random|
+-----------+--------------------+-------+--------------------+
only showing top 4 rows



**Query #2:**

Let's find top 10 users who are fan of ***Rihanna***

In [21]:
q2 = listening_df.select('user_id').filter(listening_df.artist == 'Rihanna').groupby(
    'user_id').agg( count('user_id').alias('count') ).orderBy(desc('count')).limit(10) 
q2.show()

+---------------+-----+
|        user_id|count|
+---------------+-----+
|        thiessu|  179|
|    eyessetkyle|  166|
|           adxx|  164|
|missnumberthree|  156|
|helloiamnatalie|  128|
|          nmjnb|  124|
|        AndyyyA|  123|
|     BIGBANG186|  121|
|     mixedvibes|  120|
|       AndyKitt|  115|
+---------------+-----+



**Query #3:**

find top 10 famous tracks 

In [22]:
q3 = listening_df.select('artist','track').groupby('artist','track').agg(count('*').alias('count')
).orderBy(desc('count')).limit(10)
q3.show()

+--------------+----------------+-----+
|        artist|           track|count|
+--------------+----------------+-----+
| Justin Bieber|           Sorry| 3381|
|Arctic Monkeys|Do I Wanna Know?| 2865|
|      Bon Iver|     Skinny Love| 2836|
|          Zayn|      PILLOWTALK| 2701|
|   The Killers|   Mr Brightside| 2690|
|       Rihanna|            Work| 2646|
|      Bastille|         Pompeii| 2606|
|Mumford & Sons| Little Lion Man| 2520|
|Mumford & Sons|        The Cave| 2485|
| Justin Bieber|   Love Yourself| 2481|
+--------------+----------------+-----+



**Query #4:**

find top 10 famous tracks of ***Rihanna*** 

In [23]:
q4 = listening_df.select('artist','track').filter(listening_df.artist=='Rihanna' ).groupby('artist','track').agg(
    count('*').alias('count')
).orderBy(desc('count')).limit(10)
q4.show()

+-------+--------------------+-----+
| artist|               track|count|
+-------+--------------------+-----+
|Rihanna|                Work| 2646|
|Rihanna|Only Girl (In the...| 1749|
|Rihanna|We Found Love (fe...| 1575|
|Rihanna|                 S&M| 1307|
|Rihanna|            Rude Boy| 1303|
|Rihanna|            Diamonds| 1224|
|Rihanna|      Kiss it Better|  945|
|Rihanna| Where Have You Been|  844|
|Rihanna|Cheers (Drink to ...|  697|
|Rihanna|           Needed Me|  679|
+-------+--------------------+-----+



**Query #5:**

find top 10 famous albums 

In [24]:
q5 = listening_df.groupby('artist','album').agg(
    count('*').alias('count')
).orderBy(desc('count')).limit(10)
q5.show()

+--------------------+--------------------+-----+
|              artist|               album|count|
+--------------------+--------------------+-----+
|          Kanye West|   The Life Of Pablo|22310|
|              The xx|                  xx|14195|
|      Arctic Monkeys|                  AM|14090|
|               alt-J|     An Awesome Wave|13635|
|      Mumford & Sons|        Sigh No More|13543|
|      Arctic Monkeys|Whatever People S...|12731|
|            Bon Iver|            For Emma|11994|
|              Grimes|          Art Angels|11655|
|Florence + the Ma...|               Lungs|11362|
|               Adele|                  21|11215|
+--------------------+--------------------+-----+



# Task 4 :
importing the ***genre.csv*** file:

In [25]:
genre_csv_path  = '/content/drive/MyDrive/dataset/genre.csv'
genre_df = spark.read.format('csv').option('inferSchema', True).option(
    'header', True ).load(genre_csv_path)
genre_df.show()

+--------------------+-----+
|              artist|genre|
+--------------------+-----+
|                Muse| rock|
|             Nirvana| rock|
|            Bon Jovi| rock|
|          The Police| rock|
|                Kiss| rock|
|       Guns N' Roses| rock|
|         Rusted Root| rock|
|Katrina and the W...|  pop|
|         The Beatles| rock|
|        Hall & Oates|  pop|
|        Otis Redding| soul|
|         Marvin Gaye| soul|
|     The Cranberries| rock|
|            Survivor| rock|
|       Fleetwood Mac|blues|
|           Radiohead| rock|
|                Toto| rock|
|                  U2| rock|
|Creedence Clearwa...| rock|
|                 REM| rock|
+--------------------+-----+
only showing top 20 rows



let's check the data

In [26]:
listening_df.show(3)

+-----------+--------------------+--------+-----------+
|    user_id|               track|  artist|      album|
+-----------+--------------------+--------+-----------+
|000Silenced|           Price Tag|Jessie J|Who You Are|
|000Silenced|Price Tag (Acoust...|Jessie J|  Price Tag|
|000Silenced|Be Mine! (Ballad ...|   Robyn|   Be Mine!|
+-----------+--------------------+--------+-----------+
only showing top 3 rows



Let's inner join these two data frames

In [27]:
data = listening_df.join(genre_df, how='inner', on=['artist'])
data.show(6)

+--------+-----------+--------------------+-----------+--------------------+
|  artist|    user_id|               track|      album|               genre|
+--------+-----------+--------------------+-----------+--------------------+
|Jessie J|000Silenced|           Price Tag|Who You Are| Ariana Grande & ...|
|Jessie J|000Silenced|           Price Tag|Who You Are| Jhene Aiko & Rixton|
|Jessie J|000Silenced|           Price Tag|Who You Are| Nicki Minaj & Ar...|
|Jessie J|000Silenced|           Price Tag|Who You Are|                 pop|
|Jessie J|000Silenced|Price Tag (Acoust...|  Price Tag| Ariana Grande & ...|
|Jessie J|000Silenced|Price Tag (Acoust...|  Price Tag| Jhene Aiko & Rixton|
+--------+-----------+--------------------+-----------+--------------------+
only showing top 6 rows



**Query #6**

find top 10 users who are fan of ***pop*** music

In [28]:
q6 = data.select('user_id').filter(data.genre =='pop').groupby(
    'user_id').agg( count('*').alias('count') ).orderBy(desc('count')).limit(10)
q6.show()

+---------------+-----+
|        user_id|count|
+---------------+-----+
|        01Green|  496|
|      momousagi|  400|
|          mrpsb|  400|
|   BlueKnockOut|  378|
|    musicboy80s|  376|
|  incultojurgis|  374|
| ElektricOrchid|  370|
|foreign_fanatic|  350|
|   Kevin_Soutar|  346|
|  landrover2171|  301|
+---------------+-----+



**Query #7**

find top 10 famous genres

In [30]:
q6 = data.select('genre').groupby(
    'genre').agg( count('*').alias('count') ).orderBy(desc('count')).limit(10)
q6.show()

+----------+-------+
|     genre|  count|
+----------+-------+
|      rock|2691934|
|       pop|1544747|
|electronic| 551509|
|   hip hop| 532984|
|      folk| 438174|
|indie rock| 431439|
|      punk| 380915|
|       r&b| 344101|
|     metal| 208107|
|     indie| 206726|
+----------+-------+



# Task 5:
**Query #8**

find out each user favourite genre

In [32]:
q6 = data.select('user_id','genre').groupby(
    'user_id','genre').agg( count('*').alias('count_each') ).groupby('user_id','genre').agg(
        max('count_each'))
q6.show()

+---------------+--------------------+---------------+
|        user_id|               genre|max(count_each)|
+---------------+--------------------+---------------+
|          _0015|     psychedelic pop|              3|
|        01Green|          Maty Noyes|              1|
|      01srainey|            Big Sean|              2|
|      03swalker|                soul|              4|
|        0502008|       Keys N Krates|              2|
|       07berobe|national socialis...|             14|
|       07berobe|        gothic metal|              8|
|          0rdos|              celtic|              1|
|     0xenolith0|       drum and bass|              1|
|10bobrevolution|         000 Fathers|              4|
| 11shadesofgrey|          electronic|              6|
| 11shadesofgrey|             baroque|              1|
|      1337fatal|                jazz|             14|
|        14M5K0T|                jazz|             70|
|          16-19| June Carter & Me...|              1|
|         

**Query #9**

find out how many pop,rock,metal and hip hop singers we have

and then visulize it using bar chart 

Now, let's visualize the results using ***matplotlib***

now lets visualize these two lists using a bar chart